In [ ]:
#!apt install cmake

In [ ]:
!git clone https://github.com/cloner174/mr.DGA.git

In [ ]:
%cd mr.DGA

In [1]:
import pandas as pd
import os

In [2]:
# path to data

data_path = 'data/input/'

In [3]:
# Load data

name_of_batches = os.listdir(data_path)

data_batches_path = []
for any_batch_name in name_of_batches :
    
    path_to_this_batch = os.path.join(data_path, any_batch_name)
    data_batches_path.append(path_to_this_batch)

data_batches_path.sort()
print('data_batches_path : \n' , data_batches_path)
data_batches = []
for path_to_this_batch in data_batches_path :
    
    temp_batch = pd.read_csv(path_to_this_batch, compression='zip')
    data_batches.append(temp_batch)

print('Start to Concatation the Data Batches ')

data = pd.concat(data_batches, axis=0, ignore_index=True)
#data = pd.read_csv(data_path)
print('Concatation the Data Batches was Seccessfully Done! ')

data_batches_path : 
 ['data/input/data_batch1.csv', 'data/input/data_batch2.csv', 'data/input/data_batch3.csv', 'data/input/data_batch4.csv', 'data/input/data_batch5.csv', 'data/input/data_batch6.csv', 'data/input/data_batch7.csv', 'data/input/data_batch8.csv']
Start to Concatation the Data Batches 
Concatation the Data Batches was Seccessfully Done! 


In [4]:
data.head(), '\n', data.shape

(   emotion                                             pixels
 0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
 1        0  151 150 147 155 148 133 111 140 170 174 182 15...
 2        2  231 212 156 164 174 138 161 173 182 200 106 38...
 3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
 4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,
 '\n',
 (35887, 2))

In [ ]:
%pip install imutils opencv-python dlib opencv-python-headless pillow matplotlib mediapipe

In [5]:
import cv2
import numpy as np

In [6]:
def strings_with_number_inside( data,
                                    column_range_starts : int = 0,
                                    column_range_ends : int = None,
                                    seperator = None):
        start = column_range_starts
        end = int( data.shape[1] ) if column_range_ends is None else column_range_ends
        new_data = {}
        if isinstance( data , pd.DataFrame ) :
            for i in range( data.shape[0] ) :
                temp_new_data = []
                for j in range( start, end ) :
                    cells_real_values = np.array(data.iloc[i, j].split(sep = seperator), dtype='uint8')
                    temp_new_data.append(cells_real_values)
                new_data[i] = temp_new_data

            return new_data
        else:
            if isinstance( data , np.array ) :
                pass
            else:
                raise TypeError( " data just could be  PandasDataFrame  or  NumpyArray  ! ")

In [7]:
data_pixels = strings_with_number_inside(data, column_range_starts = 1)

In [8]:
# path to save images

face_directory = 'data/output/pic/Faces-from-3.6.1'
seperated_faces_directory = 'data/output/pic/seperated-Faces-from-3.6.1'

In [9]:
# each row corresponds to an image
image_size = (48, 48)  #dimensions of image

In [ ]:
#   #       # #   #   #       # ##   #       # ##   #       # ##   #       # ##   #       # ##   #       # #

In [10]:
print('path to faces folder: ' ,face_directory,'\n')

path to faces folder:  data/output/pic/Faces-from-3.6.1 



In [11]:
#path_to_your_haarcascade_face_model_from_opencv

path_to_haarcascade_face_model = 'models/openCV/haarcascade_frontalface_default.xml'

In [12]:
face_cascade = cv2.CascadeClassifier(path_to_haarcascade_face_model)

In [13]:
os.makedirs(face_directory, exist_ok=True)

# loop to process images
for index, row in data_pixels.items():
    image_array = np.array(row, dtype=np.uint8).reshape(image_size[0], image_size[1])
    faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
      pass
    else:
        cv2.imwrite(os.path.join(face_directory, f'face_{index}.jpg'), image_array)

print("Valid Faces detecting and saving completed.")

Face cropping and saving completed.


In [15]:
face_folder = os.listdir(face_directory)

print('Number of pictures with good face that have been detected by model : ',len(face_folder))

Number of pictures with good face that have been detected by model :  8707


In [16]:
import re
images_emo_map = {}
for any_ in face_folder :
  temp_path = re.search('\d+', any_)
  temp_num = int(temp_path.group())
  images_emo_map[any_] = {}
  images_emo_map[any_]['path'] = os.path.join(face_directory, any_)
  images_emo_map[any_]['emotion'] = data.loc[temp_num, 'emotion']
  images_emo_map[any_]['iloc'] = temp_num


In [17]:
len(images_emo_map.keys()) == len(face_folder)

True

In [18]:
import subprocess
from main import Utilities

In [20]:
base_image_dir = seperated_faces_directory
labels = data['emotion'].unique()

In [21]:
# directories for each label
directories = Utilities.create_directories(base_image_dir, labels)

In [22]:
directories

{0: 'data/output/pic/seperated-Faces-from-3.6.1/0',
 2: 'data/output/pic/seperated-Faces-from-3.6.1/2',
 4: 'data/output/pic/seperated-Faces-from-3.6.1/4',
 6: 'data/output/pic/seperated-Faces-from-3.6.1/6',
 3: 'data/output/pic/seperated-Faces-from-3.6.1/3',
 5: 'data/output/pic/seperated-Faces-from-3.6.1/5',
 1: 'data/output/pic/seperated-Faces-from-3.6.1/1'}

In [23]:
# Process each image in the dataframe
for index, row in directories.items():
   for images_name in face_folder :
     if images_emo_map[images_name]['emotion'] == index :
      subprocess.call(['cp', images_emo_map[images_name]['path'], row])

In [24]:
c = 0
for indx, any_ in directories.items():
  temp_listdir = os.listdir(any_)
  print(f'the number of valid images with emotion {indx} is :', len(temp_listdir))
  c += len(temp_listdir)
print('\n --> ',c)
print( '-->' ,c == len(face_folder))

the number of valid images with emotion 0 is : 1169
the number of valid images with emotion 2 is : 948
the number of valid images with emotion 4 is : 931
the number of valid images with emotion 6 is : 1917
the number of valid images with emotion 3 is : 2680
the number of valid images with emotion 5 is : 897
the number of valid images with emotion 1 is : 165

 -->  8707
--> True


In [25]:
valid_rows = []
for indx in images_emo_map.keys():

  valid_rows.append(images_emo_map[indx]['iloc'])
len(valid_rows)

8707

In [26]:
data_valid = data.iloc[valid_rows,:]

data_valid.shape

(8707, 2)

In [27]:
data_valid.to_csv('data/output/csv/Data-Valid-from-3.6.1.csv.zip', compression={'method':'zip', 'compresslevel':8})

In [28]:
base_image_csv = 'data/output/csv/new-from-3.6.1/'
os.makedirs(base_image_csv, exist_ok=True)
labels = data['emotion'].unique()

In [32]:
for label in labels:
    subset = data_valid[data_valid['emotion'] == label]
    print(f'\n Number of rows of valid data with emotion {label} label : ', subset.shape[0])
    file_path = os.path.join(base_image_csv, f'data-valid-label{label}.csv')
    subset.to_csv(file_path)
    print(f"Data for label {label} saved to {file_path}")



 Number of rows of valid data with emotion 0 label :  1169
Data for label 0 saved to data/output/csv/new-from-3.6.1/data-valid-label0.csv

 Number of rows of valid data with emotion 2 label :  948
Data for label 2 saved to data/output/csv/new-from-3.6.1/data-valid-label2.csv

 Number of rows of valid data with emotion 4 label :  931
Data for label 4 saved to data/output/csv/new-from-3.6.1/data-valid-label4.csv

 Number of rows of valid data with emotion 6 label :  1917
Data for label 6 saved to data/output/csv/new-from-3.6.1/data-valid-label6.csv

 Number of rows of valid data with emotion 3 label :  2680
Data for label 3 saved to data/output/csv/new-from-3.6.1/data-valid-label3.csv

 Number of rows of valid data with emotion 5 label :  897
Data for label 5 saved to data/output/csv/new-from-3.6.1/data-valid-label5.csv

 Number of rows of valid data with emotion 1 label :  165
Data for label 1 saved to data/output/csv/new-from-3.6.1/data-valid-label1.csv


In [33]:
import re
images_emo_map = []
for any_ in face_folder :
    temp_path = re.search('\d+', any_)
    temp_num = int(temp_path.group())
    col = {
        'name': any_,
        'path':os.path.join(face_directory, any_),
        'emotion' : data.loc[temp_num, 'emotion'],
        'iloc' : temp_num
    }
    images_emo_map.append(col)


In [34]:
images_emo_map = pd.DataFrame( images_emo_map )
images_emo_map.to_csv('data/output/csv/images_emo_map.csv')

In [ ]:
# Now we Have Two Diffrent Kind of Approach to gather the valid images

# Our rest of work will uses this one !

In [37]:
!zip -r 'data/output/pic/Faces-from-3.6.1.zip' 'data/output/pic/Faces-from-3.6.1/'
print('Seccessfully Created zip of faces in data/output/pic/Faces-from-3.6.1.zip ')

  adding: data/output/pic/Faces-from-3.6.1/ (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_7059.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_26452.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_31551.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_1481.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_20543.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_11470.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_31366.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_34745.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_18162.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_16496.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_3020.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_3639.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3.6.1/face_16699.jpg (stored 0%)
  adding: data/output/pic/Faces-from-3

In [36]:
!zip -r 'data/output/pic/seperated-Faces-from-3.6.1.zip' 'data/output/pic/seperated-Faces-from-3.6.1/'
print('Seccessfully Created zip of faces in data/output/pic/seperated-Faces-from-3.6.1.zip ')

  adding: data/output/pic/seperated-Faces-from-3.6.1/ (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/ (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_19057.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_22502.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_23389.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_5.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_19912.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_3887.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_9483.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_6730.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_6801.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2/face_10442.jpg (stored 0%)
  adding: data/output/pic/seperated-Faces-from-3.6.1/2